<a href="https://colab.research.google.com/github/helya02/Diffusion-Model/blob/main/Diffusion_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# ---------------------------
# Diffusion hyperparameters
# ---------------------------
T = 200  # number of diffusion steps
beta = torch.linspace(1e-4, 0.02, T)  # linear noise schedule
alpha = 1. - beta
alpha_bar = torch.cumprod(alpha, dim=0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ---------------------------
# Utility: Time Channel
# ---------------------------
def get_time_channel(t, shape):
    # t is a tensor of shape (batch,)
    # Normalize t to [0,1] and expand spatially.
    t_norm = (t.float() / T).view(-1, 1, 1, 1)
    return t_norm.expand(-1, shape[1], shape[2], shape[3])

# ---------------------------
# q_sample: forward diffusion process
# ---------------------------
def q_sample(x0, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x0)
    # Index alpha_bar at time t for each sample and reshape for broadcasting.
    sqrt_alpha_bar = torch.sqrt(alpha_bar[t]).view(-1, 1, 1, 1).to(x0.device)
    sqrt_one_minus_alpha_bar = torch.sqrt(1 - alpha_bar[t]).view(-1, 1, 1, 1).to(x0.device)
    return sqrt_alpha_bar * x0 + sqrt_one_minus_alpha_bar * noise

# ---------------------------
# Dataset for Super-Resolution
# ---------------------------
# We define a custom dataset that returns:
#   - a high-res image (32×32, normalized to [-1,1])
#   - a “low-res” image: downsampled to 16×16 then upsampled back to 32×32.
class CIFAR10_SR(CIFAR10):
    def __init__(self, root, train=True, transform=None, download=False, low_res_size=16):
        super().__init__(root=root, train=train, transform=transform, download=download)
        self.low_res_size = low_res_size
        self.low_res_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((low_res_size, low_res_size), interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.Resize((32, 32), interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x * 2 - 1)  # scale [0,1] -> [-1,1]
        ])

    def __getitem__(self, index):
        # High-res image using the given transform
        img, label = super().__getitem__(index)
        # Create low-res version
        low_res_img = self.low_res_transform(img)
        return img, low_res_img

# Transform: convert image to tensor and scale to [-1,1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x * 2 - 1)
])

# Load CIFAR-10 training dataset
train_dataset = CIFAR10_SR(root='./data', train=True, transform=transform, download=True, low_res_size=16)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

# ---------------------------
# Conditional Diffusion Model
# ---------------------------
# Our model takes three inputs:
#   - x: the noisy high-res image (3 channels)
#   - low_res: the low-res image (3 channels)
#   - t: the time step (added as an extra channel)
# We concatenate these along the channel dimension (3+3+1=7 channels total).
class ConditionalDiffusionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(7, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 3, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x, low_res, t):
        # Create a time channel from t
        t_channel = get_time_channel(t, x.shape)
        # Concatenate along channels: [x (3), low_res (3), t_channel (1)] → (7 channels)
        x_in = torch.cat([x, low_res, t_channel], dim=1)
        h = self.relu(self.conv1(x_in))
        h = self.relu(self.conv2(h))
        h = self.relu(self.conv3(h))
        out = self.conv4(h)
        return out

model = ConditionalDiffusionModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ---------------------------
# Training Loop
# ---------------------------
epochs = 5  # For demonstration; increase epochs for better results.
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (high_res, low_res) in enumerate(train_loader):
        high_res = high_res.to(device)  # Ground truth image
        low_res = low_res.to(device)    # Condition: blurry low-res image
        batch_size = high_res.size(0)
        # Sample random timesteps for each image
        t = torch.randint(0, T, (batch_size,), device=device)
        noise = torch.randn_like(high_res)
        x_noisy = q_sample(high_res, t, noise)
        # Model predicts the noise added
        noise_pred = model(x_noisy, low_res, t)
        loss = F.mse_loss(noise_pred, noise)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# ---------------------------
# Sampling: Super-Resolution Generation
# ---------------------------
@torch.no_grad()
def sample_super_resolution(model, low_res, steps=T):
    # low_res: conditioned low-res image [B, 3, 32, 32]
    model.eval()
    batch_size = low_res.size(0)
    # Start from pure Gaussian noise
    x = torch.randn(batch_size, 3, 32, 32).to(device)
    for t in reversed(range(steps)):
        t_tensor = torch.full((batch_size,), t, device=device, dtype=torch.long)
        pred_noise = model(x, low_res, t_tensor)
        alpha_t = alpha[t]
        alpha_bar_t = alpha_bar[t]
        beta_t = beta[t]
        # Reverse diffusion update
        x = (1 / torch.sqrt(alpha_t)) * (x - ((1 - alpha_t) / torch.sqrt(1 - alpha_bar_t)) * pred_noise)
        if t > 0:
            noise = torch.randn_like(x)
            x = x + torch.sqrt(beta_t) * noise
    return x

# ---------------------------
# Visualize Results
# ---------------------------
# Load test dataset
test_dataset = CIFAR10_SR(root='./data', train=False, transform=transform, download=True, low_res_size=16)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=2)
high_res_test, low_res_test = next(iter(test_loader))
high_res_test = high_res_test.to(device)
low_res_test = low_res_test.to(device)

# Generate super-resolved samples
samples = sample_super_resolution(model, low_res_test)

# Create a grid to display:
# - Top row: low-res (condition)
# - Middle row: model output (super-resolved)
# - Bottom row: ground truth high-res
grid = torchvision.utils.make_grid(torch.cat([low_res_test, samples, high_res_test], dim=0), nrow=16, normalize=True)
plt.figure(figsize=(12, 6))
plt.imshow(np.transpose(grid.cpu().numpy(), (1, 2, 0)))
plt.axis('off')
plt.show()